In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
today

datetime.date(2022, 1, 7)

In [29]:
format_dict = {
               'qty':'{:,}','SHARES':'{:,}','available_qty':'{:,}','volbuy':'{:,}','volsell':'{:,}','volbal':'{:,}',
               'price':'{:,.2f}','PRICE':'{:,.2f}','max_price':'{:,.2f}','min_price':'{:,.2f}',
               'buy_target':'{:,.2f}','sell_target':'{:,.2f}','fee':'{:,.2f}','vat':'{:,.2f}',
               'net':'{:,.2f}','profit':'{:,.2f}','cost':'{:,.2f}','volume':'{:,.2f}','beta':'{:,.2f}',
               'Q4':'{:,.3f}','Q3':'{:,.3f}','Q2':'{:,.3f}','Q1':'{:,.3f}','DIVIDEND':'{:,.3f}',
               'percent':'{:,.2f}%','PERCENT':'{:,.2f}%','yearly':'{:,.2f}%','pct':'{:,.2f}%',
              }

### Tables in the process

In [30]:
# Sells table in MySql portfolio database
sql = """
SELECT T.name, B.date AS buy_date, S.* 
FROM sells S
JOIN buys B ON S.buy_id = B.id
JOIN stocks T ON B.stock_id = T.id
ORDER BY S.id DESC
LIMIT 1"""
sells = pd.read_sql(sql, conpf)
sells.style.format(format_dict)

,name,buy_date,id,buy_id,date,price,fee,vat,net,days,profit,percent,yearly,sequence,chart,dividend_id
0,IMH,2021-12-08,898,1091,2022-01-07,16.80,52.16,3.65,"25,144.19",30,"8,607.64",52.05%,633.30%,102,None,0


In [31]:
# Buys table in MySql portfolio database
names = sells['name']
name = names.to_string(index=False)

sql = """
SELECT T.name, B.* FROM buys B
JOIN stocks T ON B.stock_id = T.id
ORDER BY B.id DESC
LIMIT 1"""
print(sql)

buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)


SELECT T.name, B.* FROM buys B
JOIN stocks T ON B.stock_id = T.id
ORDER BY B.id DESC
LIMIT 1


,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,SYNEX,1092,113,2022-01-07,"6,000",31.50,391.23,27.39,Active,"189,418.62",DTD,None


In [32]:
# Buys table in MySql portfolio database
sql = """
SELECT T.name, B.* FROM buys B
JOIN stocks T ON B.stock_id = T.id
ORDER BY B.id DESC
LIMIT 1"""
print(sql)

buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)


SELECT T.name, B.* FROM buys B
JOIN stocks T ON B.stock_id = T.id
ORDER BY B.id DESC
LIMIT 1


,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,SYNEX,1092,113,2022-01-07,"6,000",31.50,391.23,27.39,Active,"189,418.62",DTD,None


In [33]:
# Stocks table in MySql portfolio database
names = buys['name']
name = names.to_string(index=False)

sql = """
SELECT T.* FROM stocks T
WHERE name = '%s'"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conpf)
stocks.style.format(format_dict)


SELECT T.* FROM stocks T
WHERE name = 'SYNEX'


,id,name,category_id,website
0,113,SYNEX,2,


In [34]:
# Buy table in MySql stock database
names = sells['name']
name = names.to_string(index=False)

sql = """
SELECT * FROM buy
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.style.format(format_dict)


SELECT * FROM buy
WHERE name = 'IMH'



,name,date,volbuy,price,volsell,volbal,active,dividend,period,grade
0,IMH,2021-12-08,"3,000.0",11.00,0.0,0.0,1,None,3,A3


In [7]:
# Dividend table in MySql stock database
names = sells['name']
name = names.to_string(index=False)

sql = """
SELECT * FROM dividend
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.style.format(format_dict)


SELECT * FROM dividend
WHERE name = 'IMH'



,NAME,Q4,Q3,Q2,Q1,DIVIDEND,PRICE,PERCENT,SHARES,XDATE,PAIDDATE,KIND,ACTUAL


### Sell transactions

### Begin of sell process for buy table in mysql stock database

In [36]:
# Sells table in MySql portfolio database
sql = """
SELECT T.name, B.date AS buy_date, B.qty, S.* 
FROM sells S
JOIN buys B ON S.buy_id = B.id
JOIN stocks T ON B.stock_id = T.id
ORDER BY S.id DESC
LIMIT 1"""
sells = pd.read_sql(sql, conpf)
sells.style.format(format_dict)

,name,buy_date,qty,id,buy_id,date,price,fee,vat,net,days,profit,percent,yearly,sequence,chart,dividend_id
0,IMH,2021-12-08,"1,500",898,1091,2022-01-07,16.80,52.16,3.65,"25,144.19",30,"8,607.64",52.05%,633.30%,102,None,0


In [37]:
names = sells['name']
name = names.to_string(index=False)
name

'IMH'

In [38]:
sr_qty = sells['qty']
qty_str = sr_qty.to_string(index=False)
qty = int(qty_str) * -1
qty

-1500

In [39]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy


SELECT * 
FROM buy 
WHERE name = 'IMH'



,name,date,volbuy,price,volsell,volbal,active,dividend,period,grade
0,IMH,2021-12-08,3000.0,11.0,0.0,0.0,1,None,3,A3


In [21]:
sqlUpd = '''
UPDATE buy
SET volbuy = volbuy + %s
WHERE name = "%s"'''
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)

rp = const.execute(sqlUpd)
rp.rowcount


UPDATE buy
SET volbuy = volbuy + -1500
WHERE name = "IMH"


1

In [15]:
sqlDel = '''
DELETE FROM buy
WHERE volbuy = 0'''
rp = const.execute(sqlDel)
rp.rowcount

0

### End of sell process to buy table in mysql stock database

### Begin of sell process to stocks table in sqlite port_lite database

In [22]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'IMH'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,684,IMH,16.80,2.08,I,12.00,15.40,142.80,0.33,11.00,"1,500",-12,22,"4,500",0,0,40%,mai


In [17]:
sqlUpd = '''
UPDATE stocks
SET available_qty = available_qty + %s
WHERE name = "%s"'''
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)

rp = conlite.execute(sqlUpd)
rp.rowcount


UPDATE stocks
SET available_qty = available_qty + -1500
WHERE name = "IMH"


1

### End of sell process to stocks table in sqlite port_lite database

### Begin of Update table dividend in mysql stock database for sells transactions

In [19]:
sql = """
SELECT * 
FROM dividend 
WHERE name = "%s" 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = "IMH" 



,NAME,Q4,Q3,Q2,Q1,DIVIDEND,PRICE,PERCENT,SHARES,XDATE,PAIDDATE,KIND,ACTUAL


In [ ]:
sqlUpd = '''
UPDATE dividend
SET shares = shares + %s
WHERE name = "%s"'''
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)

rp = const.execute(sqlUpd)
rp.rowcount

In [ ]:
sqlDel = """
DELETE FROM dividend
WHERE name = '%s' AND shares = 0
"""
sqlDel = sqlDel % name
print(sqlDel)

rp = const.execute(sqlDel)
rp.rowcount

### End of Update table dividend in mysql stock database for sells transactions

### Buys transactions

### Begin of buy process to buy table in MySQL database

In [2]:
sql = """
SELECT * 
FROM buys
ORDER BY id DESC
LIMIT 1
"""
print(sql)
buys = pd.read_sql(sql, conpf)
buys


SELECT * 
FROM buys
ORDER BY id DESC
LIMIT 1



,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,1092,113,2022-01-07,6000,31.5,391.23,27.39,Active,189418.62,DTD,None


### In case there is the same stock name in port

In [ ]:
new_cost = 0  # new average cost must be calculated manually beforehand

sqlUpd = '''
UPDATE buy
SET volbuy = volbuy + %s, price = %s
WHERE name = "%s"'''
sqlUpd = sqlUpd % (qty, new_cost, name)
print(sqlUpd)

In [ ]:
rp = const.execute(sqlUpd)
rp.rowcount

### In case of totally new stock

In [7]:
sqlIns = """
INSERT INTO buy 
VALUES('SYNEX','2022-01-07',6000,31.5,0,0,1,0,'3','A2')
"""
print(sqlIns)


INSERT INTO buy 
VALUES('SYNEX','2022-01-07',6000,31.5,0,0,1,0,'3','A2')



In [8]:
rp = const.execute(sqlIns)
rp.rowcount

1

### End of buy process to buy table in mysql stock database

### Begin of buy process to stocks table in sqlite port_lite database

In [9]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'SYNEX'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,603,SYNEX,36.50,14.90,X,31.25,0.00,174.54,1.73,24.50,"6,000",-10,15,0,1,12,DFM10%,SET100


In [21]:
sqlUpd = """
UPDATE stocks
SET available_qty = 6000, cost = 31.50, sell_target = 34.75, status = 'I', reason = '10pct'
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (name)
print(sqlUpd)


UPDATE stocks
SET available_qty = 6000, cost = 31.50, sell_target = 34.75, status = 'I', reason = '10pct'
WHERE name = 'SYNEX'



In [22]:
rp = conlite.execute(sqlUpd)
rp.rowcount

1

In [23]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'SYNEX'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,603,SYNEX,36.50,14.90,I,31.25,34.75,174.54,1.73,31.50,"6,000",-10,15,"6,000",1,12,10pct,SET100


### End of buy process to stocks table in sqlite port_lite database

### Begin of Update table dividend in mysql stock database for buy transactions

In [24]:
sql = """
SELECT * 
FROM dividend 
WHERE name = "%s" 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = "SYNEX" 



,NAME,Q4,Q3,Q2,Q1,DIVIDEND,PRICE,PERCENT,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,SYNEX,0.400,0.000,0.180,0.000,0.580,0.00,0.00%,0.0,2021-09-03,2021-09-03,,0


In [27]:
sqlUpd = '''
UPDATE dividend
SET shares = 6000
WHERE name = "%s"'''
sqlUpd = sqlUpd % (name)
print(sqlUpd)

rp = const.execute(sqlUpd)
rp.rowcount


UPDATE dividend
SET shares = 6000
WHERE name = "SYNEX"


1

In [28]:
sql = """
SELECT * 
FROM dividend 
WHERE name = "%s" 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = "SYNEX" 



,NAME,Q4,Q3,Q2,Q1,DIVIDEND,PRICE,PERCENT,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,SYNEX,0.400,0.000,0.180,0.000,0.580,0.00,0.00%,"6,000.0",2021-09-03,2021-09-03,,0


### End of Update table dividend in mysql stock database for buy transactions